In [1]:
# Core
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.multiprocessing

import glob

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from horizons import make_sim_horizons
from rebound_sim import make_sim_planets
from rebound_integrate import integrate_numpy, integrate_df, integrate_mjds, integrate_mjds_df
from planets_test import get_integration_diff, live_integration_diff
from db_config import db_engine
from db_utils import sp2df, get_columns, df2csv, csv2db, csvs2db

In [3]:
# Typing
from typing import Tuple

In [4]:
# Make sim with planets
epoch = 59000
sim_epoch = make_sim_planets(epoch=epoch)

In [5]:
# Date range to process
mjd0: int = epoch-100
mjd1: int = epoch+100

In [6]:
# Shared time_step and save_step
steps_per_day: int = 1

# Flags for building simulation archive
save_elements: bool = True
progbar: bool = True

In [7]:
# Save simulation as Numpy arrays
# body_ids, body_names, epochs, q, v, elts = integrate_numpy(sim_epoch=sim_epoch, mjd0=mjd0, mjd1=mjd1, time_step=time_step, save_elements=save_elements, progbar=True)

In [8]:
mjds = np.arange(58900, 59101)
body_ids, body_names, q, v, elts = integrate_mjds(sim_epoch=sim_epoch, mjds=mjds, save_elements=True, progbar=True)

  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
df = integrate_df(sim_epoch=sim_epoch, mjd0=mjd0, mjd1=mjd1, interval_p=1, interval_q=1, 
                  save_elements=save_elements, progbar=True)

  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
mean_dq_rel = live_integration_diff(df)
print(f'Mean relative error: {mean_dq_rel:5.3e}')

Mean relative error: 7.448e-08


In [14]:
df2 = integrate_mjds_df(sim_epoch=sim_epoch, mjds=mjds, save_elements=True, progbar=True)

  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
df2

,TimeID,BodyID,MJD,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f,M
0,84816000,10,58900,-0.004221,0.007326,0.000035,-0.000008,-0.000002,2.283442e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84816000,1,58900,-0.217196,0.256225,0.039910,-0.027079,-0.017177,1.080142e-03,0.387097,0.205652,0.122239,0.843105,0.509335,0.927129,0.627633
2,84816000,2,58900,0.047415,0.725334,0.006907,-0.020251,0.001344,1.186871e-03,0.723328,0.006747,0.059246,1.337354,0.957278,-0.795348,-0.785747
3,84816000,399,58900,-0.873491,0.478566,0.000017,-0.008494,-0.015196,1.518513e-06,1.000768,0.017385,0.000077,2.886969,-1.065969,0.823827,0.798538
4,84816000,301,58900,-0.872089,0.476317,-0.000083,-0.007999,-0.014910,-4.543185e-05,0.002577,0.046343,0.089001,1.691851,1.293892,-3.997685,2.213882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,85104000,5,59100,2.307919,-4.578456,-0.032642,0.006645,0.003755,-1.642408e-04,5.203839,0.048653,0.022751,1.754343,-1.511475,-1.346253,-1.252190
2207,85104000,6,59100,4.973013,-8.670475,-0.047221,0.004529,0.002761,-2.283492e-04,9.580985,0.051417,0.043393,1.982609,-0.402522,-2.629856,-2.577759
2208,85104000,7,59100,15.628033,12.134521,-0.157395,-0.002441,0.002923,4.248123e-05,19.198881,0.045545,0.013446,1.293179,1.708384,-2.341829,-2.274924
2209,85104000,8,59100,29.391047,-5.583021,-0.562374,0.000565,0.003103,-7.691335e-05,30.240421,0.011554,0.030885,2.299663,-2.012439,-0.474894,-0.464409


In [13]:
df

,TimeID,BodyID,MJD,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f,M
0,84816000,10,58900.0,-0.004221,0.007326,0.000035,-0.000008,-0.000002,2.283442e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84816000,1,58900.0,-0.217196,0.256225,0.039910,-0.027079,-0.017177,1.080142e-03,0.387097,0.205652,0.122239,0.843105,0.509335,0.927129,0.627633
2,84816000,2,58900.0,0.047415,0.725334,0.006907,-0.020251,0.001344,1.186871e-03,0.723328,0.006747,0.059246,1.337354,0.957278,-0.795348,-0.785747
3,84816000,399,58900.0,-0.873491,0.478566,0.000017,-0.008494,-0.015196,1.518513e-06,1.000768,0.017385,0.000077,2.886969,-1.065969,0.823827,0.798538
4,84816000,301,58900.0,-0.872089,0.476317,-0.000083,-0.007999,-0.014910,-4.543185e-05,0.002577,0.046343,0.089001,1.691851,1.293892,-3.997685,2.213882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,85104000,5,59100.0,2.307919,-4.578456,-0.032642,0.006645,0.003755,-1.642408e-04,5.203839,0.048653,0.022751,1.754343,-1.511475,-1.346253,-1.252190
2207,85104000,6,59100.0,4.973013,-8.670475,-0.047221,0.004529,0.002761,-2.283492e-04,9.580985,0.051417,0.043393,1.982609,-0.402522,-2.629856,-2.577759
2208,85104000,7,59100.0,15.628033,12.134521,-0.157395,-0.002441,0.002923,4.248123e-05,19.198881,0.045545,0.013446,1.293179,1.708384,-2.341829,-2.274924
2209,85104000,8,59100.0,29.391047,-5.583021,-0.562374,0.000565,0.003103,-7.691335e-05,30.240421,0.011554,0.030885,2.299663,-2.012439,-0.474894,-0.464409


In [16]:
df2==df

,TimeID,BodyID,MJD,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f,M
0,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2207,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2208,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2209,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
